# Mobile Robotics Project - Final Report

Group Members:
- ESCOYEZ, Antoine Jacques Richard, 335564
- POUSSIN, Jean-Baptiste Marie Alexandre, 303127
- KOCKISCH, Matthias Hugues Jörg, 303000
- STUBER, Lukas, 289304

Professor:

Prof. MONDADA, Francesco


EPFL, 11.12.2022
Course MICRO-452 Basics of Mobile Robotics

# Project Description
The goal of this project of Basics Mobile Robotics project was the control of a two wheeled Thymio robot. The environment of the robot is a map designed by the group members. On the map there needs to be obstacles into which the robot must not drive. According to the map and certain fixed obstacles present on the map, a global path from the start to the goal must be calculated using computer vision from an external camera. The start is the initial location of the Thymio, the goal can be placed by the user. On its path to the goal the robot needs to be able to detect and avoid unforeseen obstacles that are added manually. For better tracking of the path some kind of filtering must be used to improve the state estimation of the robot. The camera image serves as measurement for the latter.

# Detailed Description of our Choices
Our group chose the map to be a parking space with two centered rows of parking spaces. The yellow lines mark the edges of these spaces and are the fixed obstacles used in the global path planning. The robot is neither allowed to cross the yellow lines nor leave the map while following this path. The camera is fixed above the map and oversees the entire situation (robot, lines, free spaces, occupied spaces). To facilitate the detection of the position and orientation of the robot in the images, we stuck two coloured stickers on the top of the robot, right above its wheels. The left sticker is green, the right one is blue. The computer vision algorithm uses these two points to determine the center of the wheels axis and the orientation of the robot. The goal is a free parking space indicated by a red paper square. The unexpected obstacles (ignored by the computer vision) are other Thymio robots. The estimated state correction is done with the help of a Kalman filter whose measurement is the location of the Thymio in the camera images.

<img src="Map.jpg" width="300">

# Computer Vision
We use an *AUKEY 1080P - 30 fps* camera as the input for the computer vision. It is fixed on a stand (a lamp) and sees the entire map from above. Its position does not change during the operation of the robot.

To enable a clear and stable detection of the objects on the map, a series of adjustments are done before the object detection. The image processing is done in HSV representation of the image. The procedure is automatically started when the main function is executed. It includes:
- Adjust the luminosity <font color='red'>????</font>
- Detection of the outer borders of the map
- Adjust the red upper and lower thresholds <font color='red'>????</font>
- Adjust the green upper and lower thresholds <font color='red'>????</font>
- Adjust the blue upper and lower thresholds <font color='red'>????</font>
- Adjust the yellow upper and lower thresholds <font color='red'>????</font>

<font color='red'>Describe the steps of the computer vision to find the colorized map.</font>

The computer vision algorithm abstracts the picture of the map leaving only the relevant objects in solid colours: map surface (black), robot (green and blue spots), obstacle lines (yellow), goal (red).

# Global Path Planning
The global path planning is done using the A* algorithm, as seen in Exercise 5 of the course. It uses the simplified map generated by the computer vision part as input. The typical size of this image is <font color='red'>????</font>, which is far too large for the A* algorithm to finish in an acceptable time. To reduce the size of the image, a convolutional filter is applied to the pixels. Its size (the variable ``kernel``) can be chosen by the user. The filter divides the image from the vision part into square patches of side length $2\:\times$ ``kernel`` $+\:1$. The output pixel is 0 if:
- all the pixels in the patch are black (free space)
- at least one of the pixels is green or blue (robot position)
- at least one of the pixels is red (goal position)
The output is 1 if:
- at least one of the pixels is yellow (obstacle line)
The result of this filtering is a map of typical size 60 by 120 pixels (depending on the size of the input map) where the free, start and goal space is 0 and the obstacle space is 1.

This reduced map and the start and goal positions are fed into the A* algorithm that calculates the optimal global path from the start to the goal. The calculated path is a series of coordinates in the reduced map format. To serve as directions for the Thymio, these coordinates need to be resized to the original map (multiplied by the reduction factor $2\:\times$ ``kernel`` $+\:1$).

# Motion Control
The class ThymioControl in the file ``control.py`` solely handles the interactions between the PC and the Thymio through the ``tdmclient`` library, and contains the related higher-level algorithms for path following and obstacle avoidance. It thus controls the speeds of the robot’s wheels depending on the robot coordinates given by the Kalman filtering module, and then sends the speeds it applied back to the latter.

```python
def __init__(self):
    self.client = ClientAsync()
    self.node = aw(self.client.wait_for_node())
    aw(self.node.lock())
    self.position = [0,0]
    self.angle = 0
    self.speed_target = (0,0)
# path following
    self.stop_planned = False
    self.path = [(0,0)]
    self.path_index = 0
# obstacle avoidance
    self.proxs = np.zeros(5)
    self.obst_direction = 0
# timers
    self.stop_timer = Timer(0, self.stop)
    self.move_timer = RepeatedTimer(MOVE_INTERVAL, self.navigation)
```

The function ``navigation()``, controlling both path following and obstacle avoidance, is called by the RepeatedTimer ``move_timer`` at a period ``MOVE_INTERVAL``. This function first reads the values of the proximity sensors of the Thymio, and then starts either an obstacle avoidance step if a significant value is read in fron tin the robot, or a path following step if not. This switch is controlled by the value of ``PROX_THRESHOLD``, which is set too 1000, or around 65 millimeters, for our application. The timer interval ``MOVE_INTERVAL`` is set to ``0.21`` seconds to leave time for both fetching of the sensor values and setting new motor values, both having a set duration of 100 milliseconds. ``move_timer`` is started by the function ``follow_path()``, which is called from the main program.

```python
def navigation(self):
    self.get_prox()
    if np.any(self.proxs > PROX_THRESHOLD):
        self.avoid_obstacles()
    else: self.move_to_goal()

def follow_path(self):
    self.move_timer.start()
```

The most basic movement function is ``move(l_speed, r_speed)`` which simply sends the desired speeds to the Thymio and records the current speed-target. This communication with the robot has a standard duration of 100 milliseconds, which makes an efficient PID controller running on the computer impossible. A PID running on the Thymio itself was also not desirable due to the frequent sending of observed positions and fetching of estimated positions, requiring event-listeners on both machines, adding more delays to the movement. Running a PID and odometry on the Thymio itself is further impeded by the robot's inability to calculate with floating point values (running odometry on the computer instead would require speed-updates at a very high frequency, which is completely impossible due to the previous reasons).

We thus decided to greatly reduce the amount of communications by setting a constant speed for all movements and estimating its duration; the commands sent to the robot are now only the initial start of the movement, and the stopping command after a set duration. In this case, no (relevant) code is flashed onto the Thymio.

```python
def move(self, l_speed, r_speed=None):
    if r_speed is None: r_speed = l_speed
    aw(self.node.set_variables({"motor.left.target": [int(l_speed)],
                "motor.right.target": [int(r_speed)]}))
    self.speed_target = (l_speed, r_speed)

```

### Path following
The simple path following algorithm function ``move_to_goal()`` can be reduced to the following pseudo-code:

<pre>
calculate distance to goal  
if distance is above threshold:  
    calculate angle to goal
    if angle is above threshold:
        <i>re-orient robot</i>
    else:
        <i>move robot</i>
else:
    increment goal index in path array
    if index surpasses the length of the path:
        stop the movement timer
</pre>

Both italic lines represent a movement of the robot. For these functions, the movement duration is first estimated using the real-world speed conversion constant ``SPEED_TO_MMS``, which has been experimentally found to be $0.32\:(mm/s)^{-1}$ (at our chosen speed of ``STANDARD_SPEED = 200``, the Thymio moves 32 centimeters in 5 seconds). This duration is capped at 0.5 seconds to allow re-orientation in case of an asynchronous Kalman update.

The distance threshold ``DIST_TOL`` determines the point at which a goal is reached, while the angle threshold ``ANGLE_TOL`` triggers a re-orientation of the robot on its way to a goal. Their values were set to 10 millimeters and 0.1 rad = 5.73° respectively, which allows precise enough movement and good path following without any issues.

```python
def move_to_goal(self):
    if self.stop_planned: return
    goal = self.path[self.path_index]
    dist = math.sqrt((goal[0] - self.position[0])**2 + (goal[1] - self.position[1])**2)
    if dist > DIST_TOL:
        angle = (math.atan2(goal[1] - self.position[1], goal[0] - self.position[0])
                    - self.angle + math.pi) % (2*math.pi) - math.pi
        if abs(angle) > ANGLE_TOL:
            direction = 1 if angle > 0 else -1 # 1 = turn left, -1 = turn right
            t = abs(angle)*WHEEL_DIST / (2*STANDARD_SPEED*SPEED_TO_MMS)
            t = min(t, MAX_TIME) # allow Kalman updates
            self.timed_move(direction*STANDARD_SPEED, -direction*STANDARD_SPEED, t)
        else:
            t = dist / (STANDARD_SPEED*SPEED_TO_MMS)
            t = min(t, MAX_TIME) # allow Kalman updates
            self.timed_move(STANDARD_SPEED, STANDARD_SPEED, t)
    else:
        self.path_index += 1
        if self.path_index >= len(self.path): self.end_path()
```

The function shown above lacks a few lines at its beginning, which handle the end of the obstacle avoidance procedure; these will be shown and addressed in the corresponding section.

The final move duration is used to program ``stop_timer``, which simply stops the robot and resets the boolean ``stop_planned`` to allows re-entry into ``move_to_goal()``. The timer is programmed by the function ``timed_move()``, which also sets ``stop_planned`` that prohibits any new command from ``move_to_goal()``. Since the ``stop()`` function needs to send new motor-targets to the Thymio, the communication delay is first subtracted from the timer duration. 

```python
def stop(self):
    self.stop_planned = False
    self.move(0)
    
def timed_move(self, l_speed, r_speed, time):
    self.stop_planned = True
    self.stop_timer = Timer(time - 0.1, self.stop)  # compensate stopping delay
    self.move(l_speed, r_speed)
    self.stop_timer.start()
```

The ``end_path()`` function at the very end of ``move_to_goal()`` stops the robot and the movement timers. It then is the only function in the project that sends Aseba-code to the Thymio, which crudely plays the refrain of the electronic track "Crab Rave" by producer "Noisestorm" until the center button is pressed.

```python
def end_path(self):
    self.move_timer.stop()
    self.stop_timer.cancel()
    self.stop()
    self.crab_rave()
    
def crab_rave(self):
    program = '''
var note[19] = [2349, 1976, 1568, 1568, 2349, 2349, 1760, 1397, 1397, 2349, 2349, 1760, 1397, 1397, 2094, 2094, 1319, 1319, 1397]
var duration[19] = [2, 2, 2, 1, 2, 1, 2, 2, 1, 2, 1, 2, 2, 1, 2, 2, 2, 1, 2]
var i = 1
var play = 1
call sound.freq(note[0]/2, 8*duration[0])
onevent sound.finished
if play == 1 then
    call sound.freq(note[i]/2, 8*duration[i])
    i = i + 1
    if i == 19 then
        i = 0
    else
    end
end
onevent button.center
play = 0
    '''
    async def prog():
        await self.node.compile(program)
        await self.node.run()
    self.client.run_async_program(prog)
```

The last function of the class, called ``keyboard()``, allows the user to control the Thymio's wheel speeds using their computer's WASD keys, albeit with the usual 100 millisecond delay between input and action. This code was used to control another Thymio on the map during testing and the demostration video.

### Obstacle avoidance
In case an obstacle is detected by the front proximity sensors, ``navigation()`` calls the function ``avoid_obstacles()``. This function first stops ``stop_timer`` and its boolean in case the path following function has activated it, then calculates the speeds of both wheels using a dot product on coefficients. These coefficients make the robot turn if the obstacle is detected on the sides, and move backwards when it lies right in front of the Thymio (since the readings are practically never symmetrical, the middle coefficient has never led to a draw by repetition). Furthermore, if the obstacle is only found at the extremities of the sensed space, the robot will keep moving forward at half its standard speed. The side on which the obstacle has been found is then saved in ``obst_direction``, the speed sent to the robot, and the function starts anew while ``navigation()`` detects a significant presence in front of the robot.

```python
def avoid_obstacles(self):
    self.stop_planned = False
    self.stop_timer.cancel()
    speed = np.dot(self.proxs, [[3, -3], [1, -1], [-1, -1], [-1, 1], [-3, 3]])/100
    if (np.sum(self.proxs[1:4]) < 30): # if no obstacle in front, move forward
       speed[0] += STANDARD_SPEED/2; speed[1] += STANDARD_SPEED/2
    self.obst_direction = 1 if speed[0] < speed[1] else -1 # go left when obstacle on left and vice versa
    self.move(speed[0], speed[1])
```

Once the obstacle disappears or is completely avoided, the robot will re-enter ``move_to_goal()`` and use the last computed value of ``obst_direction`` to spurt forward while slightly turning towards the obstacle for ``OBST_TIME = 1`` seconds, in hopes of landing beyond it. This value along with ``OBST_SPEED`` and ``OBST_TURN_SPEED`` has been calibrated for driving past another immobile Thymio. It must be noted that the yellow lines of the map are ignored for the entire avoidance sequence, and also once the path following sets in again; these markings are only used during the pathfinding to place the checkpoints the robot has to travel through. The Thymio will also keep circling around the obstacle while the latter covers its next checkpoint.

Here are the lines of code of ``move_to_goal()`` that weren't shown in the previous section:

```python
def move_to_goal(self):
    if self.stop_planned: return
    if self.obst_direction != 0: # avoid previously detected obstacle
        self.timed_move(self.obst_direction*OBST_TURN_SPEED + OBST_SPEED,
                        -self.obst_direction*OBST_TURN_SPEED + OBST_SPEED, OBST_TIME)
        self.obst_direction = 0
        return
    goal = self.path[self.path_index] # start of path following code
    ...
```

# Kalman Filter
The Kalman Filter is quite standard. It is a class with the following attributes:
- the current state estimation, a vector with (x position, y position, orientation theta)
- the current state estimation covariance matrix
- the process uncertainty matrix (constant, describes the noise of the state propagation)
- the measurement uncertainty matrix (constant, describes the noise of the measurement by the camera)
- the observation matrix (constant, maps the state to the observation of the state), since all of the states can be measured, this is an identity matrix
- the time stamp of the last time the filter was called (used for calculating the elapsed time)

```python
class Kalman:
    def __init__(self):
        # timestep for state propagation
        self.dt = None
        self.prev_time = None
        # state
        self.x = np.zeros((3, 1)) # state
        self.P = 1000*np.ones((3,3)) # state covariance
        # process noise
        self.R = np.diag([NOISE_POS_XY, NOISE_POS_XY, NOISE_POS_THETA])
        # measurement noise
        self.Q = np.diag([NOISE_MEASURE_XY, NOISE_MEASURE_XY])
        # Observation Matrix H
        self.H = np.array([[1, 0, 0],
                           [0, 1, 0],
                           [0, 0, 1]])
```

and the following methods:
- state_prop <br>
It takes as input the speeds of the two wheels during the last timestep. From that it estimates the new position and state covariance matrix and updates the corresponding attributes. Because the state propagation includes sinus and cosinus calculations, it is not a linear system. To facilitate the calculations a first order approximation is used that linearizes the system. The error of the approximation is limited because the state_prop function is called every 25ms <font color='red'>????</font>. So, the involved angle is very small and so is the error of the linearization.

```python
def state_prop(self, u):
        if self.prev_time is None: # initialisation
            self.prev_time = time.time()
            return
        u = np.array(u)
        self.dt = time.time() - self.prev_time
        self.prev_time = time.time()
        # https://ocw.mit.edu/courses/6-186-mobile-autonomous-systems-laboratory-january-iap-2005/764fafce112bed6482c61f1593bd0977_odomtutorial.pdf
        (dx, dy) = self.dt*SPEED_TO_MMS*u # left and right displacements [mm]
        da = -(dy - dx)/WHEEL_DIST # rotation angle [rad]
        dc = (dx + dy)/2 # center displacement [mm]
        (vx, vy) = SPEED_TO_MMS*u # left and right wheel speeds [mm/s]
        vt = (vx + vy)/2 # translation speed [mm/s]
        vr = -(vy - vx)/WHEEL_DIST # rotation speed [rad/s]
        #print("angle is ", self.x[2])
        sin = math.sin(self.x[2])
        cos = math.cos(self.x[2])
        # state propagation
        self.x[0] = self.x[0] + dc*cos
        self.x[1] = self.x[1] + dc*sin
        self.x[2] = (self.x[2] + da) % (2*math.pi)
        # transition function (state propagation matrix)
        A = np.array([[1, 0, -self.dt*vt*sin],
                      [0, 1,  self.dt*vt*cos],
                      [0, 0, 1]])
        # input transition matrix
        L = np.array([[self.dt*vx/2*cos, -self.dt*vy/2*cos],
                      [self.dt*vx/2*cos, -self.dt*vx/2*sin],
                      [self.dt*vx/WHEEL_DIST, -self.dt*vy/WHEEL_DIST]])
        # state covariance propagation
        self.P = A@self.P@A.T + L@self.Q@L.T
```
- state_correct <br>
It takes as input the measurement of the position and angle of the robot from the camera. It calculates the Kalman gain and corrects the state and state covariance. It takes quite some time to take an image with the camera and extract the position of the robot, which means that the correct function can only be called every 1.0s <font color='red'>????</font>.

```python
def state_correct(self, z):
        z = np.reshape(z, (3,1))
        K = self.P@self.H.T@np.linalg.inv(self.H@self.P@self.H.T + self.R) # kalman gain
        self.x = self.x + K@(z - self.H@self.x) # state
        self.P = (np.eye(3) - K@self.H)@self.P # state covariance
```

# Main Function
bla bla, maybe talk about the timers

# Conclusion
conclude conclude

### Possible improvements
- The pathfinding algorithm tends to return a jagged path, which can lead to unnecessary movements of the robot even after removing most of the intermediary points. This could be improved by adding a turning penalty to the A* algorithm.
- The obstacle avoidance is very basic, and assumes the next checkpoint is beyond the obstacle. Since this is not always the case, the robot might spurt past the obstacle and then have to turn around. The spurt could be replaced by a mixed movement, composed of both obstacle avoidance and path following.
- The path following is also very crude: it only tries to reach the next checkpoint and does not take into account the checkpoints after that. This means that the robot might overshoot the checkpoint and then turn around, even though it would be faster to carry on to the next one. Similarly, after going off-path during the obstacle avoidance, rejoining the path at a later checkpoint might be faster. This might be achieved by, for example, projecting the current position onto the path.
- The Kalman filter obviously suffers from an offset on the position due to perspective, since the reference points on the Thymio are not on the ground plane. This leads to the robot driving closer to the center of the map than it should during path following, and some Kalman corrections confusing the odometric estimations. This could be solved with this trigonometric offset being taken into account in the state correction function.

In [ ]:
from __future__ import print_function
import cv2 as cv
from parking_segmentation import *
from color_segmentation import *
from color_centroids import *
from discretize_map import *
from control import ThymioControl
from Kalman import Kalman
from constants import *
from RepeatedTimer import RepeatedTimer

# IMAGE PROCESSING
id_camera = 1
##[Parking segmentation]
corners, destination_corners = set_parking_limits(id_camera)
##[Color segmentation]
segmentation, refined_color_dict_HSV, kernels, openings = get_color_mask(id_camera, corners, destination_corners, real_size=(NOMINAL_AREA_LENGTH, NOMINAL_AREA_WIDTH))
cv.namedWindow("Segmentation Result")
cv.imshow("Segmentation Result", segmentation)
key = cv.waitKey(0)
cv.destroyWindow("Segmentation Result")
##[Thymio and objective localization]
centroids = {'goal': (0, 0), 'thymio': (0, 0), 'green': (0, 0), 'blue': (0, 0)}
theta_thymio = 0
localization = None

kalman = Kalman()
thymio = ThymioControl()

def compute_centroids():
    global centroids, theta_thymio, localization
    centroids, theta_thymio, localization = get_centroids(id_camera, corners, destination_corners, refined_color_dict_HSV, kernels, openings, prev_centroids=centroids, real_size=(NOMINAL_AREA_LENGTH, NOMINAL_AREA_WIDTH), real_time=False)
    #print("centroids at", centroids['thymio'][0], centroids['thymio'][1])
    #print("thymio angle at ", theta_thymio)
    kalman.state_correct(np.array([centroids['thymio'][0], centroids['thymio'][1], theta_thymio]))
    thymio.position = (kalman.x[0, 0], kalman.x[1, 0])
    thymio.angle = kalman.x[2, 0]

def odometry():
    kalman.state_prop(thymio.speed_target)
    thymio.position = (kalman.x[0, 0], kalman.x[1, 0])
    thymio.angle = kalman.x[2, 0]
    #print(thymio.position[0], thymio.position[1], thymio.angle*180/math.pi)

# def plot_localization():
#     global localization
#     cv.namedWindow("Localization Result")
#     cv.imshow("Localization Result", localization)
#     key = cv.waitKey(30)
#     cv.destroyWindow("Localization Result")

# initialise position and path
compute_centroids()
kalman.set_state((centroids['thymio'][0], centroids['thymio'][1], theta_thymio))
thymio.position = (centroids['thymio'][0], centroids['thymio'][1])
thymio.angle = theta_thymio
path = discretize_map(segmentation, centroids)
thymio.set_path(path)

# start updating position and follow path
image_timer = RepeatedTimer(1.5, compute_centroids)
odometry_timer = RepeatedTimer(ODOMETRY_INTERVAL, odometry)
image_timer.start()
odometry_timer.start()
thymio.follow_path()